<a href="https://colab.research.google.com/github/Michwynn/London-Airbnb-Analysis---2/blob/Elias/Airbnb_Data_Base_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Configuration and library set-up**

In [1]:
# data manipulation
import pandas as pd
import numpy as np 
import re # regex
import random
from collections import Counter, defaultdict

# machine learning
from sklearn.dummy import DummyClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

# timeit
from tqdm import tqdm

# data visualisation
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

# set up working directory
import os
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Airbnb_Milestone2

# supress warnings
import warnings 
warnings.filterwarnings('ignore')

# Display all columns
pd.set_option('display.max_columns', None)

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1wUOfFY-ki2nFzneeaTtXLEeMjaSdKrrj/Airbnb_Milestone2


In [2]:
reviews_df = pd.read_csv('Datasets/reviews.csv') # read data
reviews_df.columns = reviews_df.columns.str.strip() # remove white spaces in column headings
reviews_df.head(5)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,52228441,623723762668719111,2022-05-10,37052865,Kimberly,"Great location, and the host was very responsi..."
1,52228441,505671819125096360,2021-11-28,70830110,Mahelet,Duccio is a lovely and friendly host. From arr...
2,52228441,466510411892882382,2021-10-05,83617224,Will,Duccio is a good communicator… he was very hel...
3,52228441,604109461995958546,2022-04-13,2152541,Francesco,Not entirely compliant to the pics.<br/>Good l...
4,605617198416835367,633128504578904919,2022-05-23,45418187,Waddah,Great place and great host
